In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql import *

def mergetotarget(df:DataFrame, keycolumnsformerge:list,table):
    if spark.catalog.tableExists(table):
            merge_condition = " AND ".join([f"target.{col} = source.{col}" for col in keycolumnsformerge])
            targettable = DeltaTable.forName(spark, table)
            targettable.alias("target").merge(
                        df.alias("source"),
                        merge_condition
                        ).whenMatchedUpdateAll() \
                         .whenNotMatchedInsertAll() \
                         .execute()
    else:
        df.write.format("delta").mode("overwrite").saveAsTable(table)

def mergetotargetwithscd(sourcetable, keycolumnsformerge:list,targettable):
    sourcetable = sourcetable
    targettable = targettable
    keycolumns = keycolumnsformerge
    if spark.catalog.tableExists(targettable):
    
        target_df = spark.read.table(targettable)
        target_df = target_df.filter(col("activeflag")=='Y')
        source_df = spark.read.table(sourcetable)
        nonkeyandscdfields = [columns for columns in source_df.columns if columns not in keycolumns]
        
        oncondition = " AND ".join([f"source.{c} = target.{c}" for c in keycolumns])
        conditionforfilter = " OR ".join([f"source.{c} != target.{c}" for c in nonkeyandscdfields])
        isnullcondition = " OR ".join([f"target.{c} IS NULL" for c in keycolumns])
        insert_map = {col: f"source.{col}" for col in source_df.columns}
        insert_map.update({
        "start_date": "current_timestamp()",
        "enddate": "NULL",
        "activeflag": "'Y'"
        })

        changed_df = source_df.alias("source").join(target_df.alias("target"),expr(oncondition),"left").filter(f"{isnullcondition} or {conditionforfilter}").select("source.*")
        
        changed_df = changed_df.withColumn("mergekey",concat_ws('-',*keycolumns)).union(changed_df.withColumn("mergekey",lit(None)))
        
        targetdelta = DeltaTable.forName(spark, targettable)
        ####
        targetmergekeyexp  = f"concat_ws('-', {', '.join([f'target.{c}' for c in keycolumns])})"
        targetdelta.alias("target").merge(changed_df.alias("source"),f"source.mergekey = {targetmergekeyexp}").whenMatchedUpdate(condition = "target.activeflag = 'Y'",set = {
            "activeflag":"'N'",
            "enddate": "current_timestamp()"
            }).whenNotMatchedInsert(values = insert_map).execute()
    else:
        source_df = spark.read.table(sourcetable)
        initial_df = source_df.withColumn("start_date", current_timestamp()) \
                            .withColumn("enddate", lit(None).cast("timestamp")) \
                            .withColumn("activeflag", lit("Y"))
        initial_df.write.format("delta").mode("overwrite").saveAsTable(targettable)

